In [2]:
import pandas as pd
import numpy as np

In [3]:
df= pd.read_csv('data/activ.csv')
df.head()

,name,first_name,last_name,email,first_seen,user_api_id,tags,revenue,event_approved_count,event_checked_in_count,membership_name,membership_status
0,Cristian Gian Pieer Sanchez Martell,Phayro,Martell Falcon,phayromartellfalcon380@gmail.com,2025-11-05T18:02:18.311Z,usr-XaU09ItB6pz2X9C,NaN,"0,00 US$",1,0,NaN,NaN
1,Alex Palomo,Alex,Palomo,alexpalomo@legionhack.com,2025-10-29T04:31:23.026Z,usr-4EDx7IlDtKsuOhs,NaN,"0,00 US$",0,0,NaN,NaN
2,Laura Hernandez,Laura,Hernandez,laurar1803@gmail.com,2025-10-06T16:52:48.018Z,usr-GthLF8nuwdrv4II,NaN,"0,00 US$",1,0,NaN,NaN
3,Juan Carlos Silva jaime,Juan Carlos,Silva Jaime,jcsilva@uach.mx,2025-09-28T19:16:02.033Z,usr-Fq8IHykAHIjcT1M,NaN,"0,00 US$",1,0,NaN,NaN
4,Cecilia alexandra Soto Andrade,Cecilia,Soto Andrade,sotoandrade069@gmail.com,2025-09-26T02:38:33.212Z,usr-U8jYkpT4BrtuA7k,NaN,"0,00 US$",1,0,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3696 entries, 0 to 3695
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    3422 non-null   object 
 1   first_name              2985 non-null   object 
 2   last_name               2807 non-null   object 
 3   email                   3696 non-null   object 
 4   first_seen              3696 non-null   object 
 5   user_api_id             3696 non-null   object 
 6   tags                    4 non-null      object 
 7   revenue                 3696 non-null   object 
 8   event_approved_count    3696 non-null   int64  
 9   event_checked_in_count  3696 non-null   int64  
 10  membership_name         0 non-null      float64
 11  membership_status       0 non-null      float64
dtypes: float64(2), int64(2), object(8)
memory usage: 346.6+ KB


In [5]:
# Conteo de valores vacíos en la columna 'name'
count_empty = df['name'].isna().sum()
print(f"Registros con 'name' vacío: {count_empty}")

Registros con 'name' vacío: 274


In [6]:
# Revisar emails de registros sin nombre para deducir posibles nombres
df_no_name = df[df['name'].isna() | df['name'].str.strip().eq('')]

# Extraer la parte local del email (antes del @) como posible nombre
df_no_name_sample = df_no_name[['name', 'first_name', 'email']].copy()
df_no_name_sample['email_local'] = df_no_name_sample['email'].str.split('@').str[0]

print(f"Total de registros sin nombre: {len(df_no_name_sample)}\n")
print("Primeros 20 registros con sus emails:")
print(df_no_name_sample.head(20))

# Mostrar estadísticas de los emails
print(f"\n\nTotal de emails únicos: {df_no_name_sample['email'].nunique()}")

Total de registros sin nombre: 274

Primeros 20 registros con sus emails:
    name first_name                           email           email_local
349  NaN        NaN              s.loer@outlook.com                s.loer
350  NaN        NaN               mburrola1@uach.mx             mburrola1
351  NaN        NaN             leosausix@gmail.com             leosausix
352  NaN        NaN        fernandoaginer@gmail.com        fernandoaginer
353  NaN        NaN        marlenfavela@hotmail.com          marlenfavela
356  NaN        NaN           daneliz18@hotmail.com             daneliz18
358  NaN        NaN               rocio.rojo@tec.mx            rocio.rojo
359  NaN        NaN   jberenice.mendoza@outlook.com     jberenice.mendoza
360  NaN        NaN         alejandromaju@gmail.com         alejandromaju
361  NaN        NaN        ishida.cecilia@gmail.com        ishida.cecilia
363  NaN        NaN          pablo.aguirre@meddi.mx         pablo.aguirre
364  NaN        NaN          victor.mo

In [7]:
# Crear columna email_local en el dataframe principal
df['email_local'] = df['email'].str.split('@').str[0]

In [8]:
# Limpiar la columna 'name': eliminar espacios dobles y capitalizar cada palabra
df['name'] = df['name'].str.replace(r'\s+', ' ', regex=True).str.strip().str.title()

In [9]:
import unicodedata

# Función para eliminar tildes/acentos
def remove_accents(text):
    if pd.isna(text):
        return text
    # Normalizar a NFD (descompone caracteres con tildes)
    # y eliminar marcas diacríticas
    nfd = unicodedata.normalize('NFD', text)
    return ''.join([c for c in nfd if unicodedata.category(c) != 'Mn'])

# Aplicar a la columna 'name'
df['name'] = df['name'].apply(remove_accents)

print("Tildes eliminadas exitosamente")
print(f"\nPrimeros 10 registros después de eliminar tildes:")
print(df[['name', 'first_name', 'last_name']].head(10))

Tildes eliminadas exitosamente

Primeros 10 registros después de eliminar tildes:
                                  name   first_name         last_name
0  Cristian Gian Pieer Sanchez Martell       Phayro    Martell Falcon
1                          Alex Palomo         Alex            Palomo
2                      Laura Hernandez        Laura         Hernandez
3              Juan Carlos Silva Jaime  Juan Carlos       Silva Jaime
4       Cecilia Alexandra Soto Andrade      Cecilia      Soto Andrade
5                     Stephanie Jaquez    Stephanie            Jaquez
6           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez
7                            Luis Diaz         Luis              Diaz
8           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez
9                          Johan Bader        Johan             Bader


In [10]:
# Normalizar todos los nombres: capitalizar cada palabra
df['name'] = df['name'].str.title()
df['first_name'] = df['first_name'].str.title()
df['last_name'] = df['last_name'].str.title()

# Actualizar también la columna full_name después de normalizar
df['full_name'] = df['name'].fillna('').str.strip().str.lower()

print("Nombres normalizados exitosamente")
print(f"\nPrimeros 10 registros después de la normalización:")
print(df[['name', 'first_name', 'last_name', 'full_name']].head(10))

Nombres normalizados exitosamente

Primeros 10 registros después de la normalización:
                                  name   first_name         last_name  \
0  Cristian Gian Pieer Sanchez Martell       Phayro    Martell Falcon   
1                          Alex Palomo         Alex            Palomo   
2                      Laura Hernandez        Laura         Hernandez   
3              Juan Carlos Silva Jaime  Juan Carlos       Silva Jaime   
4       Cecilia Alexandra Soto Andrade      Cecilia      Soto Andrade   
5                     Stephanie Jaquez    Stephanie            Jaquez   
6           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez   
7                            Luis Diaz         Luis              Diaz   
8           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez   
9                          Johan Bader        Johan             Bader   

                             full_name  
0  cristian gian pieer sanchez martell  
1                          a

In [11]:
# Eliminar espacios dobles en todas las columnas de tipo string
df = df.apply(lambda x: x.str.replace(r'\s+', ' ', regex=True).str.strip() if x.dtype == "object" else x)

print("Espacios dobles eliminados exitosamente")
print(f"\nPrimeros 10 registros después de limpiar espacios:")
print(df[['name', 'first_name', 'last_name', 'email']].head(10))

Espacios dobles eliminados exitosamente

Primeros 10 registros después de limpiar espacios:
                                  name   first_name         last_name  \
0  Cristian Gian Pieer Sanchez Martell       Phayro    Martell Falcon   
1                          Alex Palomo         Alex            Palomo   
2                      Laura Hernandez        Laura         Hernandez   
3              Juan Carlos Silva Jaime  Juan Carlos       Silva Jaime   
4       Cecilia Alexandra Soto Andrade      Cecilia      Soto Andrade   
5                     Stephanie Jaquez    Stephanie            Jaquez   
6           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez   
7                            Luis Diaz         Luis              Diaz   
8           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez   
9                          Johan Bader        Johan             Bader   

                                email  
0    phayromartellfalcon380@gmail.com  
1           alexpalomo@l

In [12]:
# Crear función para extraer nombre del email
def extract_name_from_email(email):
    if pd.isna(email):
        return ''
    
    # Extraer la parte local del email (antes del @)
    local_part = email.split('@')[0]
    
    # Limpiar caracteres no alfabéticos comunes
    # Mantener solo letras, espacios y puntos
    cleaned = local_part.replace('.', ' ').replace('_', ' ').replace('-', ' ')
    
    # Remover números al final (ej: maria123 -> maria)
    import re
    cleaned = re.sub(r'\d+$', '', cleaned)
    
    # Capitalizar cada palabra
    name_parts = cleaned.split()
    capitalized = ' '.join([part.capitalize() for part in name_parts if part.isalpha()])
    
    return capitalized.strip()

# Crear nueva columna con nombre extraído del email
df['name_from_email'] = df['email'].apply(extract_name_from_email)

# Mostrar registros donde 'name' está vacío pero se pudo extraer del email
mask_empty_name = df['name'].isna() | df['name'].str.strip().eq('')
df_extracted = df[mask_empty_name & (df['name_from_email'] != '')].copy()

print(f"Total de registros con nombre extraído del email: {len(df_extracted)}\n")
print("Primeros 20 registros:")
print(df_extracted[['name', 'name_from_email', 'email']].head(20))

Total de registros con nombre extraído del email: 274

Primeros 20 registros:
    name       name_from_email                           email
349  NaN                S Loer              s.loer@outlook.com
350  NaN              Mburrola               mburrola1@uach.mx
351  NaN             Leosausix             leosausix@gmail.com
352  NaN        Fernandoaginer        fernandoaginer@gmail.com
353  NaN          Marlenfavela        marlenfavela@hotmail.com
356  NaN               Daneliz           daneliz18@hotmail.com
358  NaN            Rocio Rojo               rocio.rojo@tec.mx
359  NaN     Jberenice Mendoza   jberenice.mendoza@outlook.com
360  NaN         Alejandromaju         alejandromaju@gmail.com
361  NaN        Ishida Cecilia        ishida.cecilia@gmail.com
363  NaN         Pablo Aguirre          pablo.aguirre@meddi.mx
364  NaN        Victor Morales          victor.morales@uacj.mx
365  NaN         Hectordecionr         hectordecionr@gmail.com
366  NaN  Luis Melendez Araiza  luis.mel

In [13]:
# Crear columna para identificar si first_name contiene dos nombres
df['has_two_first_names'] = df['first_name'].fillna('').str.strip().str.split().str.len() >= 2

# Crear columnas para separar los dos nombres
df['first_name_1'] = df['first_name'].fillna('').str.strip().str.split().str[0]
df['first_name_2'] = df['first_name'].fillna('').str.strip().str.split().str[1]

print(f"Registros con dos nombres: {df['has_two_first_names'].sum()}\n")
print("Primeros 20 registros con dos nombres:")
print(df[df['has_two_first_names']][['first_name', 'first_name_1', 'first_name_2', 'name']].head(20))

Registros con dos nombres: 339

Primeros 20 registros con dos nombres:
           first_name first_name_1 first_name_2  \
3         Juan Carlos         Juan       Carlos   
52     Barbara Alicia      Barbara       Alicia   
61     María Fernanda        María     Fernanda   
79      Lidia Yesenia        Lidia      Yesenia   
98      Martha Silvia       Martha       Silvia   
99      Jorge Alberto        Jorge      Alberto   
122       Mia Isabela          Mia      Isabela   
133          Xiao Yan         Xiao          Yan   
138  María De Lourdes        María           De   
141     Aldo Dionicio         Aldo     Dionicio   
147     José Humberto         José     Humberto   
152    Maria Fernanda        Maria     Fernanda   
172    Maria Cristina        Maria     Cristina   
187        Javier Noe       Javier          Noe   
189        Jesús Obed        Jesús         Obed   
198       José Carlos         José       Carlos   
205      Jesús Andrés        Jesús       Andrés   
212       A

In [14]:
# Crear columna para identificar si last_name contiene dos apellidos
df['has_two_last_names'] = df['last_name'].fillna('').str.strip().str.split().str.len() >= 2

# Crear columnas para separar los dos apellidos
df['last_name_1'] = df['last_name'].fillna('').str.strip().str.split().str[0]
df['last_name_2'] = df['last_name'].fillna('').str.strip().str.split().str[1]

print(f"Registros con dos apellidos: {df['has_two_last_names'].sum()}\n")
print("Primeros 20 registros con dos apellidos:")
print(df[df['has_two_last_names']][['last_name', 'last_name_1', 'last_name_2', 'name']].head(20))

Registros con dos apellidos: 1282

Primeros 20 registros con dos apellidos:
                     last_name last_name_1 last_name_2  \
0               Martell Falcon     Martell      Falcon   
3                  Silva Jaime       Silva       Jaime   
4                 Soto Andrade        Soto     Andrade   
6             Navarrete Jaquez   Navarrete      Jaquez   
8             Navarrete Jaquez   Navarrete      Jaquez   
13  Alejjandro Cervantes Tarín  Alejjandro   Cervantes   
15               Palomec Ortiz     Palomec       Ortiz   
17              Morales Ibarra     Morales      Ibarra   
19              Chavez Lazcano      Chavez     Lazcano   
21                Leon Mendoza        Leon     Mendoza   
22   Del Carmen Acosta Luevano         Del      Carmen   
23          David Vargas Perez       David      Vargas   
24                 Leal Chávez        Leal      Chávez   
25            Aguilar Quiñonez     Aguilar    Quiñonez   
31              Ortiz Hernádez       Ortiz    Hernádez

In [15]:
df.head(50)

,name,first_name,last_name,email,first_seen,user_api_id,tags,revenue,event_approved_count,event_checked_in_count,...,membership_status,email_local,full_name,name_from_email,has_two_first_names,first_name_1,first_name_2,has_two_last_names,last_name_1,last_name_2
0,Cristian Gian Pieer Sanchez Martell,Phayro,Martell Falcon,phayromartellfalcon380@gmail.com,2025-11-05T18:02:18.311Z,usr-XaU09ItB6pz2X9C,NaN,"0,00 US$",1,0,...,NaN,phayromartellfalcon380,cristian gian pieer sanchez martell,Phayromartellfalcon,False,Phayro,NaN,True,Martell,Falcon
1,Alex Palomo,Alex,Palomo,alexpalomo@legionhack.com,2025-10-29T04:31:23.026Z,usr-4EDx7IlDtKsuOhs,NaN,"0,00 US$",0,0,...,NaN,alexpalomo,alex palomo,Alexpalomo,False,Alex,NaN,False,Palomo,NaN
2,Laura Hernandez,Laura,Hernandez,laurar1803@gmail.com,2025-10-06T16:52:48.018Z,usr-GthLF8nuwdrv4II,NaN,"0,00 US$",1,0,...,NaN,laurar1803,laura hernandez,Laurar,False,Laura,NaN,False,Hernandez,NaN
3,Juan Carlos Silva Jaime,Juan Carlos,Silva Jaime,jcsilva@uach.mx,2025-09-28T19:16:02.033Z,usr-Fq8IHykAHIjcT1M,NaN,"0,00 US$",1,0,...,NaN,jcsilva,juan carlos silva jaime,Jcsilva,True,Juan,Carlos,True,Silva,Jaime
4,Cecilia Alexandra Soto Andrade,Cecilia,Soto Andrade,sotoandrade069@gmail.com,2025-09-26T02:38:33.212Z,usr-U8jYkpT4BrtuA7k,NaN,"0,00 US$",1,0,...,NaN,sotoandrade069,cecilia alexandra soto andrade,Sotoandrade,False,Cecilia,NaN,True,Soto,Andrade
5,Stephanie Jaquez,Stephanie,Jaquez,mario.simental0208@gmail.com,2025-09-26T01:57:45.053Z,usr-ShowEBU58V81RCy,NaN,"0,00 US$",1,0,...,NaN,mario.simental0208,stephanie jaquez,Mario Simental,False,Stephanie,NaN,False,Jaquez,NaN
6,Stephanie Navarrete Jaquez,Stephanie,Navarrete Jaquez,fannyjaquez4566@gmail.com,2025-09-26T01:53:27.884Z,usr-73xJIFmdQJNY5hK,NaN,"0,00 US$",1,0,...,NaN,fannyjaquez4566,stephanie navarrete jaquez,Fannyjaquez,False,Stephanie,NaN,True,Navarrete,Jaquez
7,Luis Diaz,Luis,Diaz,mi6141526425@gmail.com,2025-09-26T01:52:16.008Z,usr-NlNR7sHw59X7mSK,NaN,"0,00 US$",1,0,...,NaN,mi6141526425,luis diaz,Mi,False,Luis,NaN,False,Diaz,NaN
8,Stephanie Navarrete Jaquez,Stephanie,Navarrete Jaquez,stephanienavarretejaquez@gmail.com,2025-09-26T01:49:09.650Z,usr-JIVumRu4rwmJR9y,NaN,"0,00 US$",1,0,...,NaN,stephanienavarretejaquez,stephanie navarrete jaquez,Stephanienavarretejaquez,False,Stephanie,NaN,True,Navarrete,Jaquez
9,Johan Bader,Johan,Bader,maxkmp22@gmail.com,2025-09-26T01:09:06.206Z,usr-W1U23T9UAnod66J,NaN,"0,00 US$",1,0,...,NaN,maxkmp22,johan bader,Maxkmp,False,Johan,NaN,False,Bader,NaN


In [16]:
# Rellenar campos vacíos de 'name' con el valor de 'name_from_email'
# Solo para registros donde 'name' está vacío y 'name_from_email' tiene un valor válido
mask_to_fill = (df['name'].isna() | df['name'].str.strip().eq('')) & (df['name_from_email'] != '')

# Contar cuántos registros se van a actualizar
count_to_update = mask_to_fill.sum()

# Realizar la actualización
df.loc[mask_to_fill, 'name'] = df.loc[mask_to_fill, 'name_from_email']

# Actualizar también full_name para estos registros
df.loc[mask_to_fill, 'full_name'] = df.loc[mask_to_fill, 'name'].str.strip().str.lower()

print(f"Se actualizaron {count_to_update} registros con nombres extraídos del email\n")

# Verificar cuántos registros quedan con 'name' vacío después de la actualización
mask_still_empty = df['name'].isna() | df['name'].str.strip().eq('')
count_still_empty = mask_still_empty.sum()

print(f"Registros con 'name' vacío antes: {count_empty}")
print(f"Registros actualizados: {count_to_update}")
print(f"Registros con 'name' vacío después: {count_still_empty}\n")

# Mostrar algunos ejemplos de los registros actualizados
print("Primeros 20 registros actualizados:")
print(df[mask_to_fill][['name', 'name_from_email', 'email', 'full_name']].head(20))

Se actualizaron 274 registros con nombres extraídos del email

Registros con 'name' vacío antes: 274
Registros actualizados: 274
Registros con 'name' vacío después: 0

Primeros 20 registros actualizados:
                     name       name_from_email  \
349                S Loer                S Loer   
350              Mburrola              Mburrola   
351             Leosausix             Leosausix   
352        Fernandoaginer        Fernandoaginer   
353          Marlenfavela          Marlenfavela   
356               Daneliz               Daneliz   
358            Rocio Rojo            Rocio Rojo   
359     Jberenice Mendoza     Jberenice Mendoza   
360         Alejandromaju         Alejandromaju   
361        Ishida Cecilia        Ishida Cecilia   
363         Pablo Aguirre         Pablo Aguirre   
364        Victor Morales        Victor Morales   
365         Hectordecionr         Hectordecionr   
366  Luis Melendez Araiza  Luis Melendez Araiza   
367             Yodeseoun      

In [17]:
import re

def infer_name_from_email(email_local):
    """
    Intenta inferir un nombre real separando palabras que están pegadas en el email.
    Retorna cadena vacía si no puede inferir con confianza o si parece un nombre genérico.
    """
    if pd.isna(email_local) or not email_local:
        return ''
    
    # Limpiar números y caracteres especiales
    cleaned = re.sub(r'[0-9._-]', ' ', email_local).strip()
    
    # Lista de palabras que indican que NO es un nombre de persona
    skip_words = [
        'contacto', 'info', 'admin', 'soporte', 'support', 'ventas', 
        'comunicacion', 'asistente', 'assistant', 'ayuda', 'help',
        'registro', 'hola', 'hello', 'test', 'prueba', 'gerencia'
    ]
    
    # Si contiene alguna palabra genérica, retornar vacío
    if any(word in cleaned.lower() for word in skip_words):
        return ''
    
    # Si es muy corto (menos de 4 caracteres), probablemente no sea un nombre completo
    if len(cleaned) < 4:
        return ''
    
    # Intentar separar palabras pegadas usando mayúsculas como delimitador
    # Ejemplo: "FernandoGiner" -> "Fernando Giner"
    separated = re.sub(r'([a-z])([A-Z])', r'\1 \2', email_local)
    separated = re.sub(r'[0-9._-]', ' ', separated).strip()
    
    # Si después de separar tiene 2 o más palabras alfabéticas, es candidato
    words = [w for w in separated.split() if w.isalpha() and len(w) >= 2]
    
    # Solo aceptar si tiene al menos 2 palabras (nombre y apellido)
    if len(words) >= 2:
        return ' '.join(words).title()
    
    return ''

# Crear nueva columna con nombres inferidos mejorados
df['name_inferred'] = df['email_local'].apply(infer_name_from_email)

# Actualizar solo los registros donde 'name' sigue vacío y se pudo inferir un nombre válido
mask_to_infer = (df['name'].isna() | df['name'].str.strip().eq('')) & (df['name_inferred'] != '')

count_inferred = mask_to_infer.sum()

# Aplicar la actualización
df.loc[mask_to_infer, 'name'] = df.loc[mask_to_infer, 'name_inferred']
df.loc[mask_to_infer, 'full_name'] = df.loc[mask_to_infer, 'name'].str.strip().str.lower()

print(f"Se infirieron {count_inferred} nombres adicionales desde el email\n")

# Mostrar ejemplos de nombres inferidos
if count_inferred > 0:
    print("Ejemplos de nombres inferidos:")
    print(df[mask_to_infer][['name', 'name_inferred', 'email', 'email_local']].head(20))
else:
    print("No se pudieron inferir nombres adicionales")

# Contar cuántos registros quedan sin nombre
mask_final_empty = df['name'].isna() | df['name'].str.strip().eq('')
count_final_empty = mask_final_empty.sum()

print(f"\nRegistros con 'name' vacío después de la inferencia: {count_final_empty}")

Se infirieron 0 nombres adicionales desde el email

No se pudieron inferir nombres adicionales

Registros con 'name' vacío después de la inferencia: 0


In [18]:
df.head()

,name,first_name,last_name,email,first_seen,user_api_id,tags,revenue,event_approved_count,event_checked_in_count,...,email_local,full_name,name_from_email,has_two_first_names,first_name_1,first_name_2,has_two_last_names,last_name_1,last_name_2,name_inferred
0,Cristian Gian Pieer Sanchez Martell,Phayro,Martell Falcon,phayromartellfalcon380@gmail.com,2025-11-05T18:02:18.311Z,usr-XaU09ItB6pz2X9C,NaN,"0,00 US$",1,0,...,phayromartellfalcon380,cristian gian pieer sanchez martell,Phayromartellfalcon,False,Phayro,NaN,True,Martell,Falcon,
1,Alex Palomo,Alex,Palomo,alexpalomo@legionhack.com,2025-10-29T04:31:23.026Z,usr-4EDx7IlDtKsuOhs,NaN,"0,00 US$",0,0,...,alexpalomo,alex palomo,Alexpalomo,False,Alex,NaN,False,Palomo,NaN,
2,Laura Hernandez,Laura,Hernandez,laurar1803@gmail.com,2025-10-06T16:52:48.018Z,usr-GthLF8nuwdrv4II,NaN,"0,00 US$",1,0,...,laurar1803,laura hernandez,Laurar,False,Laura,NaN,False,Hernandez,NaN,
3,Juan Carlos Silva Jaime,Juan Carlos,Silva Jaime,jcsilva@uach.mx,2025-09-28T19:16:02.033Z,usr-Fq8IHykAHIjcT1M,NaN,"0,00 US$",1,0,...,jcsilva,juan carlos silva jaime,Jcsilva,True,Juan,Carlos,True,Silva,Jaime,
4,Cecilia Alexandra Soto Andrade,Cecilia,Soto Andrade,sotoandrade069@gmail.com,2025-09-26T02:38:33.212Z,usr-U8jYkpT4BrtuA7k,NaN,"0,00 US$",1,0,...,sotoandrade069,cecilia alexandra soto andrade,Sotoandrade,False,Cecilia,NaN,True,Soto,Andrade,


In [19]:
# Cuenta registros con 'name' vacío (NaN o solo espacios)
mask_empty_name = df['name'].isna() | df['name'].str.strip().eq('')
count_empty_name = int(mask_empty_name.sum())
print(count_empty_name)

0


In [20]:
# Identificar y analizar registros duplicados basados en 'full_name'
duplicated_names = df[df['full_name'] != '']['full_name'].value_counts()
duplicated_names = duplicated_names[duplicated_names > 1]

print(f"Total de nombres duplicados: {len(duplicated_names)}")
print(f"Total de registros afectados: {duplicated_names.sum()}\n")

# Mostrar los nombres más duplicados
print("Top 20 nombres más duplicados:")
print(duplicated_names.head(20))

# Mostrar detalles de algunos registros duplicados
if len(duplicated_names) > 0:
    print("\n" + "="*80)
    print("EJEMPLO: Primeros registros duplicados")
    print("="*80)
    
    # Tomar el primer nombre duplicado como ejemplo
    first_duplicated_name = duplicated_names.index[0]
    df_example = df[df['full_name'] == first_duplicated_name].sort_values('email')
    print(f"\nNombre: {first_duplicated_name}")
    print(f"Cantidad de duplicados: {len(df_example)}\n")
    print(df_example[['name', 'first_name', 'last_name', 'email', 'full_name']])

Total de nombres duplicados: 219
Total de registros afectados: 493

Top 20 nombres más duplicados:
full_name
contacto            6
carlos garcia       5
ventas              5
a                   5
carlos              5
alejandro           5
diego               5
alan                4
alejandro romero    4
armando             4
daniel              4
info                4
rafael gonzalez     4
andrea              3
eduardo padilla     3
jose parra          3
jaime gonzalez      3
salvador beltran    3
lizeth estrada      3
luis nevarez        3
Name: count, dtype: int64

EJEMPLO: Primeros registros duplicados

Nombre: contacto
Cantidad de duplicados: 6

          name first_name last_name                         email full_name
2037  Contacto        NaN       NaN          contacto@alsuper.com  contacto
3116  Contacto        NaN       NaN  contacto@chihuahuafutura.org  contacto
2039  Contacto        NaN       NaN            contacto@demek.com  contacto
1993  Contacto        NaN       NaN 

In [21]:
# Limpiar full_name eliminando valores genéricos que no son nombres reales
generic_values = ['contacto', 'ventas', 'a', 'info']

# Crear máscara para identificar registros con valores genéricos en full_name
mask_generic = df['full_name'].isin(generic_values)

# Contar cuántos registros se van a limpiar
count_generic = mask_generic.sum()

print(f"Registros con valores genéricos en full_name: {count_generic}\n")

# Mostrar ejemplos antes de limpiar
if count_generic > 0:
    print("Ejemplos antes de limpiar:")
    print(df[mask_generic][['name', 'full_name', 'email']].head(20))
    print("\n" + "="*80 + "\n")

# Reemplazar full_name con cadena vacía para valores genéricos
df.loc[mask_generic, 'full_name'] = ''

print(f"Se limpiaron {count_generic} registros con valores genéricos\n")

# Verificar después de la limpieza
print("Valores únicos de full_name después de limpiar:")
print(df['full_name'].value_counts().head(20))

Registros con valores genéricos en full_name: 20

Ejemplos antes de limpiar:
          name full_name                          email
1982  Contacto  contacto      contacto@grupomesteno.com
1993  Contacto  contacto           contacto@deutzmx.com
1994      Info      info         info@artecpisos.com.mx
2004    Ventas    ventas          ventas@cofiasa.com.mx
2008      Info      info                info@climas.com
2014    Ventas    ventas        ventas@grupoamex.com.mx
2018  Contacto  contacto               contacto@tr3z.mx
2021      Info      info            info@coprose.com.mx
2024    Ventas    ventas           ventas@abitat.com.mx
2035    Ventas    ventas  ventas@tacglobalsolutions.com
2037  Contacto  contacto           contacto@alsuper.com
2039  Contacto  contacto             contacto@demek.com
2051    Ventas    ventas       ventas@hotelsoberano.com
2840         A         a               a01562935@tec.mx
2854         A         a               a01569486@tec.mx
2863         A         a   

In [22]:
# Eliminar duplicados basados en 'full_name', manteniendo solo la primera ocurrencia
# Solo se eliminan duplicados donde full_name no esté vacío

# Contar duplicados antes de eliminar
duplicates_before = df[df['full_name'] != '']['full_name'].duplicated().sum()
total_before = len(df)

# Eliminar duplicados manteniendo el primer registro de cada full_name
# Los registros con full_name vacío se mantienen todos
df = df[~((df['full_name'] != '') & df['full_name'].duplicated(keep='first'))]

# Resetear el índice después de eliminar filas
df = df.reset_index(drop=True)

total_after = len(df)
duplicates_removed = total_before - total_after

print(f"Registros antes de eliminar duplicados: {total_before}")
print(f"Duplicados eliminados: {duplicates_removed}")
print(f"Registros después de eliminar duplicados: {total_after}")

# Verificar que ya no hay duplicados en full_name (excluyendo valores vacíos)
duplicates_after = df[df['full_name'] != '']['full_name'].duplicated().sum()
print(f"\nDuplicados restantes en full_name: {duplicates_after}")

# Mostrar los primeros registros del dataframe limpio
print("\nPrimeros 10 registros del dataframe limpio:")
print(df[['name', 'first_name', 'last_name', 'email', 'full_name']].head(10))

Registros antes de eliminar duplicados: 3696
Duplicados eliminados: 258
Registros después de eliminar duplicados: 3438

Duplicados restantes en full_name: 0

Primeros 10 registros del dataframe limpio:
                                  name   first_name         last_name  \
0  Cristian Gian Pieer Sanchez Martell       Phayro    Martell Falcon   
1                          Alex Palomo         Alex            Palomo   
2                      Laura Hernandez        Laura         Hernandez   
3              Juan Carlos Silva Jaime  Juan Carlos       Silva Jaime   
4       Cecilia Alexandra Soto Andrade      Cecilia      Soto Andrade   
5                     Stephanie Jaquez    Stephanie            Jaquez   
6           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez   
7                            Luis Diaz         Luis              Diaz   
8                          Johan Bader        Johan             Bader   
9                       Sophia Alarcon       Sophia           Alarcó

In [23]:
import sys, subprocess

# Instalación opcional y conteo de género estimado a partir del primer nombre
try:
    import gender_guesser.detector as gender
except ModuleNotFoundError:
    # Instalamos la librería si no está disponible
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "gender-guesser"], check=True)
    import gender_guesser.detector as gender

detector = gender.Detector(case_sensitive=False)

def guess_gender(row):
    first = row["first_name"]
    fallback = row["name"]
    candidate = (str(first) if pd.notna(first) and str(first).strip() else str(fallback)).split()[0] if pd.notna(fallback) else ""
    raw = detector.get_gender(candidate) if candidate else "unknown"
    mapping = {
        "male": "male",
        "mostly_male": "male",
        "female": "female",
        "mostly_female": "female",
        "andy": "unknown",
        "unknown": "unknown",
    }
    return mapping.get(raw, "unknown")

df["gender_guess"] = df.apply(guess_gender, axis=1)
gender_counts = df["gender_guess"].value_counts().rename_axis("gender").reset_index(name="count")
print(gender_counts)

    gender  count
0     male   1737
1   female   1186
2  unknown    515


In [24]:
# Filtrar registros con género "unknown" y mostrar los primeros 20
df_unknown = df[df["gender_guess"] == "unknown"]
print(f"Total de nombres con género desconocido: {len(df_unknown)}\n")
print(df_unknown[["name", "first_name", "gender_guess"]].head(20))

Total de nombres con género desconocido: 515

                                    name first_name gender_guess
0    Cristian Gian Pieer Sanchez Martell     Phayro      unknown
12    Gnolaum Alejjandro Cervantes Tarin    Gnolaum      unknown
14         Yarelli Abigail Palomec Ortiz    Yarelli      unknown
18                 Sandta Chavez Lazcano     Sandta      unknown
27                                 Eneas      Eneas      unknown
31                                 Zared      Zared      unknown
39                  Aricela Perez Espino    Aricela      unknown
52                         Idaly Alvarez      Idaly      unknown
53                         Rossy Navarro      Rossy      unknown
60                     Mifam El Porvenir      Mifam      unknown
64                          Jovany Diego     Jovany      unknown
74                      Amaranda Becerra   Amaranda      unknown
86                           Aide Neaves       Aidé      unknown
95                                 Areli    

In [ ]:
# Filtrar nombres femeninos y mostrar los primeros 20
df_female = df[df['gender_guess'] == 'female']
print(f"Total de nombres femeninos: {len(df_female)}\n")
print(df_female[['name', 'first_name', 'gender_guess']].head(20))

Total de nombres femeninos: 1186

                                 name      first_name gender_guess
2                     Laura Hernandez           Laura       female
4      Cecilia Alexandra Soto Andrade         Cecilia       female
5                    Stephanie Jaquez       Stephanie       female
6          Stephanie Navarrete Jaquez       Stephanie       female
9                      Sophia Alarcon          Sophia       female
16              Renata Morales Ibarra          Renata       female
20  Vanessa Del Carmen Acosta Luevano         Vanessa       female
25                          Valeria S         Valeria       female
28                       Lucia Orozco           Lucia       female
33                     Annette Aguayo         Annette       female
42            Gabriela Fuentes Garcia        Gabriela       female
45                        Yadira Pena          Yadira       female
46                      Nadia Sanchez             NaN       female
50     Barbara Alicia Bordon

In [ ]:
#imprime nombres masculinos
df_male = df[df['gender_guess'] == 'male']
print(f"Total de nombres masculinos: {len(df_male)}\n")
print(df_male[['name', 'first_name', 'gender_guess']].head(20))

Total de nombres masculinos: 1737

                               name   first_name gender_guess
1                       Alex Palomo         Alex         male
3           Juan Carlos Silva Jaime  Juan Carlos         male
7                         Luis Diaz         Luis         male
8                       Johan Bader        Johan         male
10                Christopher Bravo  Christopher         male
11                 Alejandro Arroyo    Alejandro         male
13                      Max Morales          Max         male
15                    Jaime Jimenez        Jaime         male
17                      Eric Aguayo         Eric         male
19         Manuel Ivan Leon Mendoza       Manuel         male
21        Gerardo David Vargasperez      Gerardo         male
22        Daniel Arturo Leal Chavez       Daniel         male
23  Joel Alejandro Aguilar Quinonez         Joel         male
24                     Angel Medina        Angel         male
26                        Guillermo

NOMBRES CON 2 APELLIDOS (se eliminarán duplicados):
                                        name        first_name                      last_name  num_palabras                                       email                               full_name
2862                Alejandro Romero Ornelas         Alejandro                 Romero Ornelas             3                       aljandro.34@gmail.com                alejandro romero ornelas
3156             Alondra Maria Martinez Ayon           Alondra            Maria Martinez Ayon             4                  implan@implanchihuahua.org             alondra maria martinez ayon
1665                       Alonso De La Cruz            Alonso                     De La Cruz             4                          adroal08@gmail.com                       alonso de la cruz
2100              Ana Silvia Salgado Acevedo               NaN                            NaN             4                  superanita101@gmail.com.mx              ana silvia salg

In [25]:
# Analizar más detalladamente los casos con género unknown
print("Análisis detallado de nombres con género unknown:")
print("="*60)

# Extraer el primer nombre de first_name para los casos unknown
df_unknown_analysis = df[df["gender_guess"] == "unknown"].copy()
df_unknown_analysis['first_word_from_first_name'] = df_unknown_analysis['first_name'].str.split().str[0]
df_unknown_analysis['first_word_from_name'] = df_unknown_analysis['name'].str.split().str[0]

print(f"Total de registros con género unknown: {len(df_unknown_analysis)}")
print("\nPrimeros 30 casos con first_name extraído:")
print(df_unknown_analysis[['name', 'first_name', 'first_word_from_first_name', 'first_word_from_name']].head(30))

# Ver qué primeros nombres son más comunes en los casos unknown
print("\nPrimeros nombres más comunes en casos unknown:")
first_names_unknown = df_unknown_analysis['first_word_from_first_name'].value_counts().head(20)
print(first_names_unknown)

Análisis detallado de nombres con género unknown:
Total de registros con género unknown: 515

Primeros 30 casos con first_name extraído:
                                     name  first_name  \
0     Cristian Gian Pieer Sanchez Martell      Phayro   
12     Gnolaum Alejjandro Cervantes Tarin     Gnolaum   
14          Yarelli Abigail Palomec Ortiz     Yarelli   
18                  Sandta Chavez Lazcano      Sandta   
27                                  Eneas       Eneas   
31                                  Zared       Zared   
39                   Aricela Perez Espino     Aricela   
52                          Idaly Alvarez       Idaly   
53                          Rossy Navarro       Rossy   
60                      Mifam El Porvenir       Mifam   
64                           Jovany Diego      Jovany   
74                       Amaranda Becerra    Amaranda   
86                            Aide Neaves        Aidé   
95                                  Areli       Areli   
99      

In [26]:
# Función mejorada para inferir género con listas específicas de nombres
def improved_gender_guess(row):
    """
    Función mejorada para inferir género usando listas de nombres comunes
    y múltiples estrategias de detección
    """
    
    # Listas de nombres comunes femeninos y masculinos en español
    female_names = {
        'maria', 'ana', 'carmen', 'josefa', 'isabel', 'dolores', 'pilar', 'teresa', 
        'rosa', 'francisca', 'antonia', 'mercedes', 'elena', 'cristina', 'patricia',
        'laura', 'monica', 'sandra', 'andrea', 'silvia', 'beatriz', 'marta',
        'lucia', 'sofia', 'claudia', 'natalia', 'paula', 'sara', 'adriana',
        'valeria', 'camila', 'daniela', 'alejandra', 'carolina', 'gabriela',
        'fernanda', 'mariana', 'paola', 'veronica', 'rocio', 'lorena', 'noelia',
        'esther', 'concepcion', 'amparo', 'gloria', 'esperanza', 'remedios',
        'encarnacion', 'montserrat', 'inmaculada', 'rosario', 'soledad'
    }
    
    male_names = {
        'antonio', 'manuel', 'jose', 'francisco', 'david', 'juan', 'javier',
        'jesus', 'daniel', 'carlos', 'miguel', 'rafael', 'pedro', 'angel',
        'alejandro', 'diego', 'sergio', 'fernando', 'pablo', 'jorge', 'luis',
        'alberto', 'ivan', 'oscar', 'ruben', 'adrian', 'raul', 'gonzalo',
        'mario', 'victor', 'eduardo', 'roberto', 'marcos', 'antonio', 'emilio',
        'ricardo', 'guillermo', 'andres', 'ramon', 'nicolas', 'jaime', 'salvador',
        'cesar', 'ignacio', 'francisco', 'joaquin', 'enrique', 'agustin', 'felipe'
    }
    
    first_name = str(row.get('first_name', ''))
    name = str(row.get('name', ''))
    
    # Extraer el primer nombre de first_name o name
    if first_name and first_name != 'nan' and first_name.strip():
        candidate = first_name.split()[0].lower().strip()
    elif name and name != 'nan' and name.strip():
        candidate = name.split()[0].lower().strip()
    else:
        return "unknown"
    
    # Limpiar el candidato
    candidate = candidate.replace('.', '').replace(',', '')
    
    # Verificar en listas específicas primero
    if candidate in female_names:
        return "female"
    elif candidate in male_names:
        return "male"
    
    # Si no está en las listas, usar el detector original
    try:
        raw_gender = detector.get_gender(candidate)
        mapping = {
            "male": "male",
            "mostly_male": "male", 
            "female": "female",
            "mostly_female": "female",
            "andy": "unknown",
            "unknown": "unknown",
        }
        return mapping.get(raw_gender, "unknown")
    except:
        return "unknown"

# Aplicar la función mejorada solo a los registros con género unknown
print("Aplicando función mejorada de detección de género...")
mask_unknown = df["gender_guess"] == "unknown"
print(f"Registros con género unknown antes: {mask_unknown.sum()}")

# Crear nueva columna temporal para comparar
df['gender_improved'] = df.apply(improved_gender_guess, axis=1)

# Ver cuántos se pudieron mejorar
df_comparison = df[mask_unknown].copy()
df_comparison['changed'] = df_comparison['gender_guess'] != df_comparison['gender_improved']
improved_count = df_comparison['changed'].sum()

print(f"Registros que se pudieron mejorar: {improved_count}")

# Mostrar algunos ejemplos de mejoras
if improved_count > 0:
    print("\nEjemplos de mejoras:")
    examples = df_comparison[df_comparison['changed']][['name', 'first_name', 'gender_guess', 'gender_improved']].head(20)
    print(examples)
    
    # Actualizar la columna principal
    df.loc[mask_unknown, 'gender_guess'] = df.loc[mask_unknown, 'gender_improved']
    
    # Mostrar el nuevo conteo
    print("\nNuevo conteo de géneros:")
    new_counts = df["gender_guess"].value_counts()
    print(new_counts)
else:
    print("No se encontraron mejoras con la función personalizada")

# Eliminar columna temporal
df = df.drop('gender_improved', axis=1)

Aplicando función mejorada de detección de género...
Registros con género unknown antes: 515
Registros que se pudieron mejorar: 9

Ejemplos de mejoras:
                               name       first_name gender_guess  \
147                   Rocio Ramirez            Rocio      unknown   
168         Rocio Rodriguez Sanchez            Rocio      unknown   
351                      Rocio Rojo              NaN      unknown   
875        Concepcion Ramos Caldera       Concepcion      unknown   
1109           Rocio Mendez Jimenez              NaN      unknown   
2436    Rocio Adela Flores Carrillo            Rocio      unknown   
2960  Joaquin Eliezer Blanco Chacon  Joaquin Eliezer      unknown   
3333                  Rocio Infante            Rocio      unknown   
3397                 Joaquin Galvez          Joaquin      unknown   

     gender_improved  
147           female  
168           female  
351           female  
875           female  
1109          female  
2436          femal

In [27]:
# Análisis adicional para nombres que terminan en patrones específicos
def analyze_name_patterns(name):
    """
    Analiza patrones en los nombres para inferir género
    """
    if pd.isna(name) or not name.strip():
        return "unknown"
        
    name = str(name).lower().strip()
    first_word = name.split()[0] if name.split() else ""
    
    # Patrones femeninos
    female_endings = ['a', 'ia', 'ina', 'ana', 'ela', 'isa', 'ita', 'eta']
    female_patterns = ['maria', 'ana', 'ina', 'lia', 'nda', 'tra']
    
    # Patrones masculinos  
    male_endings = ['o', 'io', 'ino', 'ano', 'elo', 'ito', 'eto']
    male_patterns = ['andro', 'ando', 'ardo', 'erto', 'urdo']
    
    # Verificar terminaciones
    for ending in female_endings:
        if first_word.endswith(ending) and len(first_word) > len(ending):
            return "female"
    
    for ending in male_endings:
        if first_word.endswith(ending) and len(first_word) > len(ending):
            return "male"
            
    # Verificar patrones internos
    for pattern in female_patterns:
        if pattern in first_word:
            return "female"
            
    for pattern in male_patterns:
        if pattern in first_word:
            return "male"
    
    return "unknown"

# Aplicar análisis de patrones a los nombres que siguen siendo unknown
print("Aplicando análisis de patrones para los nombres restantes unknown...")

mask_still_unknown = df["gender_guess"] == "unknown"
print(f"Registros que siguen siendo unknown: {mask_still_unknown.sum()}")

# Aplicar análisis de patrones
df_patterns = df[mask_still_unknown].copy()
df_patterns['pattern_gender'] = df_patterns['first_name'].apply(analyze_name_patterns)

# Si first_name no da resultado, probar con name
mask_still_unknown_after_first = df_patterns['pattern_gender'] == "unknown"
df_patterns.loc[mask_still_unknown_after_first, 'pattern_gender'] = \
    df_patterns.loc[mask_still_unknown_after_first, 'name'].apply(analyze_name_patterns)

# Ver cuántos se mejoraron con patrones
improved_by_pattern = (df_patterns['pattern_gender'] != "unknown").sum()
print(f"Registros mejorados con análisis de patrones: {improved_by_pattern}")

if improved_by_pattern > 0:
    # Mostrar ejemplos
    examples_patterns = df_patterns[df_patterns['pattern_gender'] != "unknown"]
    print("\nEjemplos de mejoras por patrones:")
    print(examples_patterns[['name', 'first_name', 'pattern_gender']].head(15))
    
    # Actualizar en el dataframe principal
    mask_to_update = mask_still_unknown & (df_patterns['pattern_gender'] != "unknown")
    df.loc[df_patterns.index[df_patterns['pattern_gender'] != "unknown"], 'gender_guess'] = \
        df_patterns.loc[df_patterns['pattern_gender'] != "unknown", 'pattern_gender']
    
    # Mostrar conteo final
    print("\nConteo final de géneros:")
    final_counts = df["gender_guess"].value_counts()
    print(final_counts)
else:
    print("No se encontraron mejoras adicionales con análisis de patrones")

Aplicando análisis de patrones para los nombres restantes unknown...
Registros que siguen siendo unknown: 506
Registros mejorados con análisis de patrones: 166

Ejemplos de mejoras por patrones:
                                    name first_name pattern_gender
0    Cristian Gian Pieer Sanchez Martell     Phayro           male
18                 Sandta Chavez Lazcano     Sandta         female
39                  Aricela Perez Espino    Aricela         female
74                      Amaranda Becerra   Amaranda         female
130           Carolina Xiao Yan Ye Liang   Xiao Yan           male
150              Columba Fernandez Gomez    Columba         female
282                        Anayeli Nieto    Anayeli         female
294                       Anahi Sandoval      Anahí         female
326              Raymundo Carrasco Leyva   Raymundo           male
343                             Mburrola        NaN         female
345                       Fernandoaginer        NaN           male
3

In [28]:
# Resumen final de la mejora en detección de género
print("RESUMEN FINAL DE MEJORAS EN DETECCIÓN DE GÉNERO")
print("="*60)
print("Registros iniciales con gender 'unknown': 515")
print("Mejorados con función personalizada: 9")
print("Mejorados con análisis de patrones: 166")
print("Total de mejoras: 175")
print("Registros que permanecen 'unknown': 340")
print(f"Porcentaje de mejora: {(175/515)*100:.1f}%")

print("\n" + "="*60)
print("DISTRIBUCIÓN FINAL DE GÉNEROS")
print("="*60)
final_distribution = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in final_distribution.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

print("\n" + "="*60)
print("ANÁLISIS DE LOS 340 CASOS QUE AÚN QUEDAN COMO 'UNKNOWN'")
print("="*60)

# Analizar los casos que siguen siendo unknown
df_final_unknown = df[df["gender_guess"] == "unknown"].copy()

print(f"Total de casos unknown restantes: {len(df_final_unknown)}")
print("\nPrimeros 20 casos que aún son unknown:")
print(df_final_unknown[['name', 'first_name', 'email']].head(20))

# Analizar patrones en los casos que quedan unknown
print("\nTipos de casos que permanecen unknown:")
print("\n1. Nombres con first_name vacío:")
empty_first_name = df_final_unknown[df_final_unknown['first_name'].isna() | 
                                   (df_final_unknown['first_name'].str.strip() == '')]
print(f"   Cantidad: {len(empty_first_name)}")
print(empty_first_name[['name', 'first_name']].head(10))

print("\n2. Nombres poco comunes o de otros idiomas:")
non_empty_first_name = df_final_unknown[~(df_final_unknown['first_name'].isna() | 
                                         (df_final_unknown['first_name'].str.strip() == ''))]
print(f"   Cantidad: {len(non_empty_first_name)}")
print(non_empty_first_name[['name', 'first_name']].head(10))

RESUMEN FINAL DE MEJORAS EN DETECCIÓN DE GÉNERO
Registros iniciales con gender 'unknown': 515
Mejorados con función personalizada: 9
Mejorados con análisis de patrones: 166
Total de mejoras: 175
Registros que permanecen 'unknown': 340
Porcentaje de mejora: 34.0%

DISTRIBUCIÓN FINAL DE GÉNEROS
MALE: 1790 registros (52.1%)
FEMALE: 1308 registros (38.0%)
UNKNOWN: 340 registros (9.9%)

ANÁLISIS DE LOS 340 CASOS QUE AÚN QUEDAN COMO 'UNKNOWN'
Total de casos unknown restantes: 340

Primeros 20 casos que aún son unknown:
                                   name first_name  \
12   Gnolaum Alejjandro Cervantes Tarin    Gnolaum   
14        Yarelli Abigail Palomec Ortiz    Yarelli   
27                                Eneas      Eneas   
31                                Zared      Zared   
52                        Idaly Alvarez      Idaly   
53                        Rossy Navarro      Rossy   
60                    Mifam El Porvenir      Mifam   
64                         Jovany Diego     Jovan

In [30]:
# NUEVO ENFOQUE AVANZADO: Inferencia de género con múltiples estrategias
print("ESTRATEGIA AVANZADA DE INFERENCIA DE GÉNERO")
print("="*60)

def advanced_gender_inference(row):
    """
    Función avanzada que combina múltiples estrategias para inferir género:
    1. Diccionario expandido de nombres en español e internacional
    2. Análisis de terminaciones más específicas
    3. Patrones lingüísticos
    4. Nombres compuestos y variaciones
    """
    
    # Diccionario expandido con más nombres femeninos
    expanded_female_names = {
        # Nombres tradicionales españoles
        'maria', 'ana', 'carmen', 'josefa', 'isabel', 'dolores', 'pilar', 'teresa',
        'rosa', 'francisca', 'antonia', 'mercedes', 'elena', 'cristina', 'patricia',
        'laura', 'monica', 'sandra', 'andrea', 'silvia', 'beatriz', 'marta',
        'lucia', 'sofia', 'claudia', 'natalia', 'paula', 'sara', 'adriana',
        'valeria', 'camila', 'daniela', 'alejandra', 'carolina', 'gabriela',
        'fernanda', 'mariana', 'paola', 'veronica', 'rocio', 'lorena', 'noelia',
        'esther', 'concepcion', 'amparo', 'gloria', 'esperanza', 'remedios',
        'encarnacion', 'montserrat', 'inmaculada', 'rosario', 'soledad',
        # Nombres modernos y variaciones
        'araceli', 'aracely', 'ariadna', 'arianna', 'azucena', 'begonia', 'belinda',
        'berenice', 'berta', 'betina', 'blanca', 'brenda', 'carla', 'caridad',
        'celeste', 'celia', 'charo', 'clara', 'concha', 'coral', 'delia',
        'diana', 'edith', 'elisa', 'elvira', 'emma', 'esperanza', 'estrella',
        'eugenia', 'eva', 'fatima', 'felisa', 'flor', 'gemma', 'gema', 'gisela',
        'gracia', 'guadalupe', 'hilda', 'ines', 'irene', 'iris', 'itziar',
        'ivonne', 'jacinta', 'jessica', 'jimena', 'julia', 'karina', 'leticia',
        'lila', 'lola', 'lourdes', 'luisa', 'luz', 'macarena', 'magdalena',
        'manuela', 'marcela', 'margarita', 'marina', 'marisol', 'marlene',
        'matilde', 'milagros', 'miriam', 'nadia', 'nancy', 'nerea', 'nieves',
        'nuria', 'olga', 'paloma', 'pamela', 'perla', 'petra', 'raquel',
        'rebeca', 'regina', 'rita', 'sabrina', 'sonia', 'susana', 'tamara',
        'tania', 'tatiana', 'trinidad', 'ursula', 'vanesa', 'vanessa', 'victoria',
        'violeta', 'viviana', 'yolanda', 'zaida', 'zoraida'
    }
    
    # Diccionario expandido con más nombres masculinos
    expanded_male_names = {
        # Nombres tradicionales españoles
        'antonio', 'manuel', 'jose', 'francisco', 'david', 'juan', 'javier',
        'jesus', 'daniel', 'carlos', 'miguel', 'rafael', 'pedro', 'angel',
        'alejandro', 'diego', 'sergio', 'fernando', 'pablo', 'jorge', 'luis',
        'alberto', 'ivan', 'oscar', 'ruben', 'adrian', 'raul', 'gonzalo',
        'mario', 'victor', 'eduardo', 'roberto', 'marcos', 'emilio',
        'ricardo', 'guillermo', 'andres', 'ramon', 'nicolas', 'jaime', 'salvador',
        'cesar', 'ignacio', 'joaquin', 'enrique', 'agustin', 'felipe',
        # Nombres modernos y variaciones
        'abraham', 'adrian', 'alan', 'alonso', 'alvaro', 'anibal', 'armando',
        'arturo', 'aurelio', 'bernardo', 'bruno', 'calixto', 'camilo', 'candido',
        'carmelo', 'cayetano', 'claudio', 'cristian', 'cristobal', 'damian',
        'dario', 'demetrio', 'domingo', 'edmundo', 'eladio', 'eliseo', 'elogio',
        'emerson', 'erick', 'ernesto', 'esteban', 'eugenio', 'evaristo', 'ezequiel',
        'fabian', 'fabricio', 'facundo', 'fausto', 'federico', 'felix', 'fermin',
        'fidel', 'florentino', 'fortunato', 'gabriel', 'gaspar', 'german', 'gilberto',
        'gino', 'gregorio', 'gustavo', 'hector', 'helio', 'hernan', 'hilario',
        'homero', 'horacio', 'hugo', 'humberto', 'isidro', 'ismael', 'israel',
        'jacinto', 'jacobo', 'jasinto', 'jeronimo', 'joaquin', 'jonas', 'jonatan',
        'jordan', 'josue', 'jovani', 'julian', 'julio', 'junior', 'justin',
        'leandro', 'leonardo', 'leopoldo', 'lorenzo', 'luciano', 'macedonio',
        'marcelo', 'mariano', 'martin', 'mateo', 'mauricio', 'maximo', 'melchor',
        'nestor', 'norberto', 'octavio', 'olegario', 'orlando', 'patricio',
        'paulino', 'primitivo', 'quirino', 'rafael', 'ramiro', 'reinaldo',
        'renato', 'reynaldo', 'rodrigo', 'rogelio', 'rolando', 'roman', 'rosendo',
        'saul', 'sebastian', 'teodoro', 'timoteo', 'tomas', 'ulises', 'valentino',
        'valerio', 'virgilio', 'waldo', 'walter', 'wilfredo', 'winston'
    }
    
    first_name = str(row.get('first_name', ''))
    name = str(row.get('name', ''))
    
    # Extraer candidato y limpiar
    candidate = ""
    if first_name and first_name != 'nan' and first_name.strip():
        candidate = first_name.split()[0].lower().strip()
    elif name and name != 'nan' and name.strip():
        candidate = name.split()[0].lower().strip()
    else:
        return "unknown"
    
    # Limpiar caracteres especiales
    candidate = candidate.replace('.', '').replace(',', '').replace('(', '').replace(')', '')
    
    # 1. Verificar en diccionarios expandidos
    if candidate in expanded_female_names:
        return "female"
    elif candidate in expanded_male_names:
        return "male"
    
    # 2. Análisis de terminaciones más específicas
    if len(candidate) >= 3:
        # Terminaciones claramente femeninas
        if candidate.endswith(('anda', 'enda', 'inda', 'onda', 'unda')):
            return "female"
        if candidate.endswith(('iana', 'yana', 'lena', 'nela', 'cela')):
            return "female"
        if candidate.endswith(('ita', 'eta', 'ata', 'ota', 'uta')):
            return "female"
        if candidate.endswith(('isa', 'esa', 'asa', 'osa', 'usa')):
            return "female"
        
        # Terminaciones claramente masculinas
        if candidate.endswith(('ando', 'endo', 'indo', 'ondo', 'undo')):
            return "male"
        if candidate.endswith(('iano', 'yano', 'leno', 'nelo', 'celo')):
            return "male"
        if candidate.endswith(('ito', 'eto', 'ato', 'oto', 'uto')):
            return "male"
        
    # 3. Patrones específicos del español
    if len(candidate) >= 4:
        # Patrones femeninos
        if any(pattern in candidate for pattern in ['mari', 'ana', 'rosa', 'luz', 'cruz']):
            return "female"
        # Patrones masculinos  
        if any(pattern in candidate for pattern in ['juan', 'jose', 'luis', 'carl']):
            return "male"
    
    # 4. Análisis de nombres extranjeros comunes
    international_female = {
        'anne', 'anna', 'anne', 'helen', 'helen', 'mary', 'elizabeth', 'lisa',
        'susan', 'jennifer', 'michelle', 'stephanie', 'amanda', 'melissa', 'sarah',
        'kimberly', 'amy', 'angela', 'emma', 'olivia', 'sophia', 'isabella',
        'mia', 'charlotte', 'amelia', 'harper', 'evelyn', 'abigail', 'emily',
        'ella', 'madison', 'scarlett', 'victoria', 'aria', 'grace', 'chloe',
        'camila', 'penelope', 'riley', 'layla', 'lillian', 'nora', 'zoe'
    }
    
    international_male = {
        'john', 'james', 'robert', 'michael', 'william', 'david', 'richard',
        'joseph', 'thomas', 'christopher', 'charles', 'daniel', 'matthew',
        'anthony', 'mark', 'donald', 'steven', 'paul', 'andrew', 'joshua',
        'kenneth', 'kevin', 'brian', 'george', 'edward', 'ronald', 'timothy',
        'jason', 'jeffrey', 'ryan', 'jacob', 'gary', 'nicholas', 'eric',
        'jonathan', 'stephen', 'larry', 'justin', 'scott', 'brandon', 'benjamin',
        'samuel', 'gregory', 'alexander', 'patrick', 'frank', 'raymond', 'jack'
    }
    
    if candidate in international_female:
        return "female"
    elif candidate in international_male:
        return "male"
    
    # 5. Como último recurso, usar el detector original
    try:
        raw_gender = detector.get_gender(candidate)
        mapping = {
            "male": "male",
            "mostly_male": "male", 
            "female": "female",
            "mostly_female": "female"
        }
        return mapping.get(raw_gender, "unknown")
    except:
        return "unknown"

# Aplicar la función avanzada solo a los registros que aún son unknown
mask_current_unknown = df["gender_guess"] == "unknown"
current_unknown_count = mask_current_unknown.sum()
print(f"Registros con género unknown actualmente: {current_unknown_count}")

# Crear columna temporal para la nueva inferencia
df['gender_advanced'] = df[mask_current_unknown].apply(advanced_gender_inference, axis=1)

# Contar mejoras
df_advanced_test = df[mask_current_unknown].copy()
df_advanced_test['gender_advanced'] = df_advanced_test.apply(advanced_gender_inference, axis=1)
advanced_improvements = (df_advanced_test['gender_advanced'] != "unknown").sum()

print(f"Registros que se pueden mejorar con estrategia avanzada: {advanced_improvements}")

if advanced_improvements > 0:
    # Mostrar ejemplos de las mejoras
    improved_examples = df_advanced_test[df_advanced_test['gender_advanced'] != "unknown"]
    print("\nEjemplos de nombres mejorados con estrategia avanzada:")
    print(improved_examples[['name', 'first_name', 'gender_advanced']].head(20))
    
    # Aplicar las mejoras al dataframe principal
    df.loc[mask_current_unknown, 'gender_guess'] = df_advanced_test['gender_advanced']
    
    # Mostrar el nuevo conteo
    print(f"\nNUEVO CONTEO DESPUÉS DE LA ESTRATEGIA AVANZADA:")
    print("="*50)
    new_gender_counts = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in new_gender_counts.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
    
    # Calcular mejora total
    remaining_unknown = (df["gender_guess"] == "unknown").sum()
    total_improved_today = 515 - remaining_unknown  # 515 era el número inicial
    improvement_percentage = (total_improved_today / 515) * 100
    
    print(f"\nRESUMEN DE MEJORAS TOTALES:")
    print("="*40)
    print(f"Unknown inicial: 515")
    print(f"Unknown actual: {remaining_unknown}")
    print(f"Total mejorado: {total_improved_today}")
    print(f"Porcentaje de mejora total: {improvement_percentage:.1f}%")
    
else:
    print("No se encontraron mejoras adicionales con la estrategia avanzada")

# Limpiar columna temporal
if 'gender_advanced' in df.columns:
    df = df.drop('gender_advanced', axis=1)

ESTRATEGIA AVANZADA DE INFERENCIA DE GÉNERO
Registros con género unknown actualmente: 328
Registros que se pueden mejorar con estrategia avanzada: 0
No se encontraron mejoras adicionales con la estrategia avanzada


In [ ]:
# ESTRATEGIA ALTERNATIVA: Análisis contextual y manual de casos específicos
print("ANÁLISIS DETALLADO DE CASOS UNKNOWN RESTANTES")
print("="*60)

# Analizar qué tipos de nombres quedan como unknown
df_unknown_remaining = df[df["gender_guess"] == "unknown"].copy()
print(f"Total de casos unknown restantes: {len(df_unknown_remaining)}")

# Extraer primeros nombres únicos de los casos unknown
unique_first_names = df_unknown_remaining['first_name'].dropna().str.split().str[0].str.lower().value_counts()
print(f"\nPrimeros nombres más frecuentes en casos unknown:")
print(unique_first_names.head(30))

print(f"\n" + "="*60)
print("CASOS ESPECÍFICOS PARA ANÁLISIS MANUAL")
print("="*60)

# Mostrar una muestra de casos unknown para análisis
sample_unknown = df_unknown_remaining.sample(min(30, len(df_unknown_remaining)), random_state=42)
print("Muestra de 30 casos unknown para análisis:")
for idx, row in sample_unknown.iterrows():
    first_name = str(row['first_name']) if pd.notna(row['first_name']) else 'N/A'
    name = str(row['name']) if pd.notna(row['name']) else 'N/A'
    print(f"  {idx}: first_name='{first_name}' | name='{name}'")

print(f"\n" + "="*60)
print("CATEGORIZACIÓN DE CASOS UNKNOWN")
print("="*60)

# Categorizar los tipos de casos unknown
categories = {
    'nombres_vacios': 0,
    'nombres_extranjeros': 0,
    'nombres_compuestos_complejos': 0,
    'apellidos_como_nombres': 0,
    'nombres_poco_comunes': 0,
    'otros': 0
}

def categorize_unknown_case(row):
    first_name = str(row['first_name']) if pd.notna(row['first_name']) else ''
    name = str(row['name']) if pd.notna(row['name']) else ''
    
    # Casos sin información útil
    if (not first_name or first_name == 'nan' or first_name.strip() == '') and \
       (not name or name == 'nan' or name.strip() == ''):
        return 'nombres_vacios'
    
    # Extraer primer nombre
    candidate = ''
    if first_name and first_name != 'nan' and first_name.strip():
        candidate = first_name.split()[0].lower()
    elif name and name != 'nan' and name.strip():
        candidate = name.split()[0].lower()
    
    # Nombres extranjeros (contienen caracteres no típicos del español)
    if any(char in candidate for char in ['k', 'w', 'x', 'y']) and len(candidate) > 3:
        return 'nombres_extranjeros'
    
    # Nombres compuestos complejos (más de 3 palabras en first_name)
    if first_name and len(first_name.split()) > 3:
        return 'nombres_compuestos_complejos'
    
    # Posibles apellidos como nombres (empiezan con mayúscula y terminan en consonante)
    if candidate and candidate[0].isupper() and candidate[-1] in 'bcdfghjklmnpqrstvwxyz':
        return 'apellidos_como_nombres'
    
    # Nombres poco comunes (longitud inusual o patrones extraños)
    if len(candidate) > 10 or len(candidate) < 3:
        return 'nombres_poco_comunes'
    
    return 'otros'

# Aplicar categorización
for idx, row in df_unknown_remaining.iterrows():
    category = categorize_unknown_case(row)
    categories[category] += 1

print("Distribución por categorías:")
for category, count in categories.items():
    percentage = (count / len(df_unknown_remaining)) * 100
    print(f"  {category}: {count} casos ({percentage:.1f}%)")

print(f"\n" + "="*60)
print("INFERENCIA BASADA EN CASOS ESPECÍFICOS IDENTIFICADOS")
print("="*60)

def contextual_gender_inference(row):
    """
    Función de inferencia basada en análisis contextual específico
    """
    first_name = str(row['first_name']) if pd.notna(row['first_name']) else ''
    name = str(row['name']) if pd.notna(row['name']) else ''
    
    # Casos específicos identificados en el análisis
    specific_cases = {
        # Nombres que pueden aparecer en los datos
        'yuliana': 'female', 'xiomara': 'female', 'wendy': 'female', 'yesica': 'female',
        'yolanda': 'female', 'ximena': 'female', 'yasmin': 'female', 'yaneth': 'female',
        'walter': 'male', 'wilson': 'male', 'winston': 'male', 'wilmer': 'male',
        'washington': 'male', 'wenceslao': 'male', 'wilfredo': 'male', 'william': 'male',
        # Nombres compuestos comunes
        'mariaelena': 'female', 'analidia': 'female', 'mariajose': 'female',
        'juancarlos': 'male', 'josemaria': 'male', 'luisalberto': 'male',
        # Variaciones con acentos o caracteres especiales
        'josé': 'male', 'maría': 'female', 'jesús': 'male', 'andrés': 'male'
    }
    
    # Extraer candidato
    candidate = ''
    if first_name and first_name != 'nan' and first_name.strip():
        candidate = first_name.split()[0].lower().strip()
    elif name and name != 'nan' and name.strip():
        candidate = name.split()[0].lower().strip()
    
    # Limpiar caracteres especiales
    candidate = candidate.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    candidate = candidate.replace('ñ', 'n').replace('.', '').replace(',', '')
    
    # Verificar en casos específicos
    if candidate in specific_cases:
        return specific_cases[candidate]
    
    # Análisis de nombres pegados (sin espacios)
    if len(candidate) > 8:
        # Buscar patrones conocidos dentro del nombre pegado
        for known_name, gender in specific_cases.items():
            if known_name in candidate and len(known_name) >= 4:
                return gender
    
    return "unknown"

# Aplicar inferencia contextual
print("Aplicando inferencia contextual...")
improved_contextual = 0

for idx, row in df_unknown_remaining.iterrows():
    inferred_gender = contextual_gender_inference(row)
    if inferred_gender != "unknown":
        df.loc[idx, 'gender_guess'] = inferred_gender
        improved_contextual += 1

print(f"Nombres mejorados con inferencia contextual: {improved_contextual}")

if improved_contextual > 0:
    print(f"\nNUEVO CONTEO FINAL:")
    final_counts = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in final_counts.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
    
    final_unknown = (df["gender_guess"] == "unknown").sum()
    total_improvement = 515 - final_unknown
    improvement_rate = (total_improvement / 515) * 100
    
    print(f"\nMEJORA TOTAL FINAL:")
    print(f"Unknown inicial: 515")
    print(f"Unknown final: {final_unknown}")
    print(f"Total mejorado: {total_improvement} ({improvement_rate:.1f}%)")
else:
    print("No se encontraron mejoras adicionales con inferencia contextual")

ANÁLISIS DETALLADO DE CASOS UNKNOWN RESTANTES
Total de casos unknown restantes: 328

Primeros nombres más frecuentes en casos unknown:
first_name
yahir         5
abril         5
hiram         4
areli         4
nahomy        4
ever          3
jahaziel      2
estefany      2
jasiel        2
denisse       2
idis          2
yamileth      2
lety          2
verenice      2
rossy         2
idaly         2
mayrim        1
joanthan      1
yefry         1
irad          1
jassef        1
christoper    1
arad          1
jhonny        1
jovany        1
mifam         1
eneas         1
aidé          1
freider       1
zared         1
Name: count, dtype: int64

CASOS ESPECÍFICOS PARA ANÁLISIS MANUAL
Muestra de 30 casos unknown para análisis:
  2664: first_name='N/A' | name='Hiei Numata'
  1202: first_name='Yahir' | name='Yahir Eduardo Balderrama Tarango'
  2708: first_name='N/A' | name='Lmendez'
  95: first_name='Areli' | name='Areli'
  895: first_name='Chandell' | name='Chandell Stone'
  2474: first_n

In [ ]:
# Buscar todos los registros que contengan "Ana Silvia Salgado Acevedo" en la columna 'name'
df_ana_silvia = df[df['name'].str.contains('Ana Silvia Salgado Acevedo', case=False, na=False)]
print(f"Total de registros encontrados: {len(df_ana_silvia)}\n")
print(df_ana_silvia[['name', 'first_name', 'last_name', 'email', 'full_name']])

Total de registros encontrados: 1

                            name first_name last_name  \
2100  Ana Silvia Salgado Acevedo        NaN       NaN   

                           email                   full_name  
2100  superanita101@gmail.com.mx  ana silvia salgado acevedo  
